In [30]:
import pandas as pd

# Carregar os dados
df = pd.read_csv('df_train.csv')

# Calcular a taxa de churn por cidade
churn_by_city = df.groupby('City')['Churn Label'].value_counts(normalize=True).unstack().fillna(0)
churn_by_city['Churn Rate'] = churn_by_city['Yes'] / (churn_by_city['Yes'] + churn_by_city['No'])

# Ordenar cidades por taxa de churn
churn_by_city_sorted = churn_by_city.sort_values(by='Churn Rate', ascending=False)
print(churn_by_city_sorted)

#Cidades com taxas de churn mais altas podem ser identificadas ao observar as 
#cidades com os maiores valores na coluna Churn Rate.
#Se houver uma cidade com taxa de churn muito maior do que a média, isso indica que ela pode ter um problema específico.

Churn Label        No  Yes  Churn Rate
City                                  
Amador City       0.0  1.0         1.0
Walnut            0.0  1.0         1.0
Squaw Valley      0.0  1.0         1.0
South Lake Tahoe  0.0  1.0         1.0
Tipton            0.0  1.0         1.0
...               ...  ...         ...
Reedley           1.0  0.0         0.0
Richgrove         1.0  0.0         0.0
Richvale          1.0  0.0         0.0
Boonville         1.0  0.0         0.0
Calpine           1.0  0.0         0.0

[1129 rows x 3 columns]


In [31]:
# Definir clientes recentes e antigos
median_tenure = df['Tenure Months'].median()
df['Customer Type'] = df['Tenure Months'].apply(lambda x: 'Recent' if x < median_tenure else 'Old')

# Calcular a taxa de churn para cada grupo
churn_by_type = df.groupby('Customer Type')['Churn Label'].value_counts(normalize=True).unstack().fillna(0)
churn_by_type['Churn Rate'] = churn_by_type['Yes'] / (churn_by_type['Yes'] + churn_by_type['No'])
print(churn_by_type)

#Se a taxa de churn é mais alta entre os clientes classificados como "Recentes"
#isso indica que os clientes que estão na empresa há menos tempo têm maior probabilidade de sair
#Se for mais alta entre os clientes "Antigos", isso indica o contrário.

Churn Label          No       Yes  Churn Rate
Customer Type                                
Old            0.868814  0.131186    0.131186
Recent         0.602689  0.397311    0.397311


In [32]:
services = ['Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies']
service_usage = df[services].apply(lambda x: (x == 'Yes').sum())
print(service_usage)

#Os serviços que têm o menor número de "Yes" são os menos utilizados
#Isso pode incluir serviços como Phone Service, Multiple Lines, Online Backup, etc
#Analise a contagem para identificar os serviços com menor adesão.

Phone Service        5401
Multiple Lines       2507
Internet Service        0
Online Security      1696
Online Backup        2044
Device Protection    2056
Tech Support         1743
Streaming TV         2274
Streaming Movies     2321
dtype: int64


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Carregar os dados
df = pd.read_csv('df_train.csv')

# Preprocessamento
df = df.drop(['CustomerID', 'Count', 'Zip Code', 'Lat Long'], axis=1)  # Remover colunas não úteis
df = pd.get_dummies(df)  # Codificar variáveis categóricas

# Separar características e rótulo
X = df.drop('Churn Label_Yes', axis=1)  # 'Churn Label_Yes' é a coluna de rótulo binário
y = df['Churn Label_Yes']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinar e avaliar o modelo de árvore de decisão
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)
y_pred_tree = decision_tree.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
report_tree = classification_report(y_test, y_pred_tree)

# Treinar e avaliar o modelo de floresta aleatória
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train, y_train)
y_pred_forest = random_forest.predict(X_test)
accuracy_forest = accuracy_score(y_test, y_pred_forest)
report_forest = classification_report(y_test, y_pred_forest)

# Resultados
print("Árvore de Decisão")
print("Precisão:", accuracy_tree)
print("Relatório de Classificação:\n", report_tree)

print("Floresta Aleatória")
print("Precisão:", accuracy_forest)
print("Relatório de Classificação:\n", report_forest)

Árvore de Decisão
Precisão: 1.0
Relatório de Classificação:
               precision    recall  f1-score   support

       False       1.00      1.00      1.00      1328
        True       1.00      1.00      1.00       468

    accuracy                           1.00      1796
   macro avg       1.00      1.00      1.00      1796
weighted avg       1.00      1.00      1.00      1796

Floresta Aleatória
Precisão: 0.9699331848552338
Relatório de Classificação:
               precision    recall  f1-score   support

       False       0.96      1.00      0.98      1328
        True       1.00      0.88      0.94       468

    accuracy                           0.97      1796
   macro avg       0.98      0.94      0.96      1796
weighted avg       0.97      0.97      0.97      1796



In [37]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# Carregar os dados e preparar X, y
# df = pd.read_csv('train.csv')
# Preprocessamento omitido por brevidade
X = df.drop('Churn Label_Yes', axis=1)
y = df['Churn Label_Yes']

# Criar o modelo
model = DecisionTreeClassifier(random_state=42)

# Aplicar validação cruzada K-Fold com K=5
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Precisão média (K-Fold com K=5):", scores.mean())

Precisão média (K-Fold com K=5): 1.0


In [38]:
from sklearn.model_selection import LeaveOneOut

# Criar o modelo
model = DecisionTreeClassifier(random_state=42)

# Configurar Leave-One-Out
loo = LeaveOneOut()
scores = cross_val_score(model, X, y, cv=loo, scoring='accuracy')
print("Precisão média (Leave-One-Out):", scores.mean())

Precisão média (Leave-One-Out): 1.0


In [39]:
from sklearn.model_selection import StratifiedKFold

# Criar o modelo
model = DecisionTreeClassifier(random_state=42)

# Configurar Stratified K-Fold com K=5
skf = StratifiedKFold(n_splits=5)
scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
print("Precisão média (Stratified K-Fold com K=5):", scores.mean())

Precisão média (Stratified K-Fold com K=5): 1.0


In [40]:
from sklearn.model_selection import TimeSeriesSplit

# Criar o modelo
model = DecisionTreeClassifier(random_state=42)

# Configurar TimeSeriesSplit com 5 divisões
tscv = TimeSeriesSplit(n_splits=5)
scores = cross_val_score(model, X, y, cv=tscv, scoring='accuracy')
print("Precisão média (TimeSeriesSplit com 5 divisões):", scores.mean())

Precisão média (TimeSeriesSplit com 5 divisões): 1.0


In [51]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Carregar os conjuntos de dados
train_df = pd.read_csv('df_train.csv')
test_df = pd.read_csv('df_test.csv')

# Armazenar CustomerID do conjunto de teste para submissão
customer_ids = test_df['CustomerID']

# Preparar o conjunto de dados de treinamento
X_train = train_df.drop(['CustomerID', 'Churn Label'], axis=1)
y_train = train_df['Churn Label'].apply(lambda x: 1 if x == 'Yes' else 0)

# Preparar o conjunto de dados de teste
X_test = test_df.drop(['CustomerID'], axis=1)

# Codificar variáveis categóricas para ambos os conjuntos de dados
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Garantir que ambos os conjuntos de dados tenham as mesmas colunas
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Padronizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [52]:
# Aplicar PCA
pca = PCA(n_components=0.95)  # Mantém 95% da variância explicada
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


In [53]:
# Treinar o modelo Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train_pca, y_train)

RandomForestClassifier(random_state=42)

In [54]:
y_pred = model.predict(X_test_pca)

In [55]:
submission_df = pd.DataFrame({
    'CustomerID': customer_ids,
    'Churn Label': y_pred
})

In [56]:
submission_df['Churn Label'] = submission_df['Churn Label'].apply(lambda x: 'Yes' if x == 1 else 'No')


In [57]:
submission_df.to_csv('submission.csv', index=False)


In [58]:
from sklearn.model_selection import GridSearchCV

# Definir os hiperparâmetros para testar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='f1', 
                           n_jobs=-1)

# Ajustar o GridSearchCV
grid_search.fit(X_train_pca, y_train)

# Melhor hiperparâmetro encontrado
print("Best parameters:", grid_search.best_params_)


Best parameters: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 50}


In [59]:
from sklearn.feature_selection import SelectFromModel

# Seleção de features com RandomForest
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42))
selector.fit(X_train_pca, y_train)

# Aplicar a seleção de features
X_train_selected = selector.transform(X_train_pca)
X_test_selected = selector.transform(X_test_pca)


In [61]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

# Treinar um modelo XGBoost
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train_pca, y_train)

# Fazer previsões
y_pred = model.predict(X_val_pca)

# Avaliar o modelo
print("Validation F1-score:", f1_score(y_val, y_pred))

/home/raulm/TraineeCIS/env/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [23:33:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


ValueError: Feature shape mismatch, expected: 5233, got 4211

In [62]:
from sklearn.model_selection import cross_val_score

# Avaliar o modelo usando validação cruzada
scores = cross_val_score(RandomForestClassifier(random_state=42), X_train_pca, y_train, cv=5, scoring='f1')
print("Cross-validated F1-score:", scores.mean())


Cross-validated F1-score: 0.1399021952239321


In [63]:
from sklearn.metrics import f1_score

# Obter probabilidades de previsão
y_prob = model.predict_proba(X_val_pca)[:, 1]

# Ajustar o threshold
threshold = 0.3  # exemplo de threshold
y_pred_adjusted = (y_prob > threshold).astype(int)

print("F1-score with adjusted threshold:", f1_score(y_val, y_pred_adjusted))

ValueError: Feature shape mismatch, expected: 5233, got 4211

In [65]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_pca, y_train)


In [66]:
from sklearn.metrics import confusion_matrix

# Matriz de Confusão
cm = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", cm)


ValueError: Found input variables with inconsistent numbers of samples: [1198, 1057]

In [67]:
submission_df = pd.DataFrame({
    'CustomerID': customer_ids,
    'Churn Label': y_pred
})

# Converter previsões binárias para 'Yes' ou 'No'
submission_df['Churn Label'] = submission_df['Churn Label'].apply(lambda x: 'Yes' if x == 1 else 'No')

# Salvar o arquivo de submissão
submission_df.to_csv('submission.csv', index=False)